In [11]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from  datetime import datetime, timedelta
import gc
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_squared_error
from math import sqrt
import sklearn

In [12]:
#Define column types for importing data
cal_dtype = {"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
              "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
              "month": "int16", "year": "int16", "snap_CA": "int8", 'snap_TX': 'int8', 'snap_WI': 'int8'}
price_dtype = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32"}

#Read sales data
dt = pd.read_csv('sales_train_validation.csv')
dt = dt.astype({col: 'int32' for col in dt.select_dtypes('int64').columns})

#Read calendar data
cal = pd.read_csv('calendar.csv', dtype = cal_dtype)
cal['date'] = pd.to_datetime(cal['date'])
for col, col_dtype in cal_dtype.items():
    if col_dtype == "category":
        cal[col] = cal[col].cat.codes.astype("int16")
        cal[col] -= cal[col].min()
cal = cal.astype({col: 'int32' for col in cal.select_dtypes('int64').columns})
cal_1 = cal[0:1913]
future = cal[0:1941]

#Aggregate sales by store_id and dept_it
dt_st_dep = dt.groupby(['store_id', 'dept_id'], as_index=False).sum()

#Create dataframe to generate proportions for last 28 days to predict individual series from aggregated series
dt_y_pred = dt[['id','dept_id','store_id', 'd_1886', 'd_1887', 'd_1888', 'd_1889', 'd_1890', 'd_1891',
               'd_1892', 'd_1893', 'd_1894', 'd_1895', 'd_1896', 'd_1897', 'd_1898',
               'd_1899', 'd_1900', 'd_1901', 'd_1902', 'd_1903', 'd_1904', 'd_1905', 'd_1906',
               'd_1907', 'd_1908', 'd_1909', 'd_1910', 'd_1911', 'd_1912', 'd_1913',]]
d_id = dt_y_pred.dept_id.unique()
d_id_lst = d_id.tolist()
st_id = dt_y_pred.store_id.unique()
st_id_lst = st_id.tolist()

#Create empty dataframe to store submission
df_sub = pd.DataFrame(columns = ['id','dept_id','store_id', 'd_1886', 'd_1887', 'd_1888', 'd_1889', 'd_1890', 'd_1891',
               'd_1892', 'd_1893', 'd_1894', 'd_1895', 'd_1896', 'd_1897', 'd_1898',
               'd_1899', 'd_1900', 'd_1901', 'd_1902', 'd_1903', 'd_1904', 'd_1905', 'd_1906',
               'd_1907', 'd_1908', 'd_1909', 'd_1910', 'd_1911', 'd_1912', 'd_1913',])

In [13]:
for s_id in st_id_lst:                #Loop over store ids
    for d_id in d_id_lst:            #Loop over department ids
        print(s_id, d_id)
        
        #Get all individual series for this aggregate series
        hob1_st1 = dt_y_pred.loc[(dt_y_pred['dept_id'] == d_id) & (dt_y_pred['store_id'] == s_id)]
        
        #Get train set values for specific d_id and s_id
        dt_st_dep_n = dt_st_dep.loc[(dt_st_dep['dept_id'] == d_id) & (dt_st_dep['store_id'] == s_id)]
        
        #Get proportions of individual series in aggregate series
        div_coeff_28 = dt_st_dep_n[['store_id','dept_id', 'd_1886', 'd_1887', 'd_1888', 'd_1889', 'd_1890', 'd_1891',
                       'd_1892', 'd_1893', 'd_1894', 'd_1895', 'd_1896', 'd_1897', 'd_1898',
                       'd_1899', 'd_1900', 'd_1901', 'd_1902', 'd_1903', 'd_1904', 'd_1905', 'd_1906',
                       'd_1907', 'd_1908', 'd_1909', 'd_1910', 'd_1911', 'd_1912', 'd_1913',]]
        div_coeff_28 = div_coeff_28.reset_index(drop=True)
        div_coeff_lst = div_coeff_28.loc[0, :].values.tolist()

        hob1_st1['d_1886'] = hob1_st1['d_1886']/div_coeff_lst[2]
        hob1_st1['d_1887'] = hob1_st1['d_1887']/div_coeff_lst[3]
        hob1_st1['d_1888'] = hob1_st1['d_1888']/div_coeff_lst[4]
        hob1_st1['d_1889'] = hob1_st1['d_1889']/div_coeff_lst[5]
        hob1_st1['d_1890'] = hob1_st1['d_1890']/div_coeff_lst[6]
        hob1_st1['d_1891'] = hob1_st1['d_1891']/div_coeff_lst[7]
        hob1_st1['d_1892'] = hob1_st1['d_1892']/div_coeff_lst[8]
        hob1_st1['d_1893'] = hob1_st1['d_1893']/div_coeff_lst[9]
        hob1_st1['d_1894'] = hob1_st1['d_1894']/div_coeff_lst[10]
        hob1_st1['d_1895'] = hob1_st1['d_1895']/div_coeff_lst[11]
        hob1_st1['d_1896'] = hob1_st1['d_1896']/div_coeff_lst[12]
        hob1_st1['d_1897'] = hob1_st1['d_1897']/div_coeff_lst[13]
        hob1_st1['d_1898'] = hob1_st1['d_1898']/div_coeff_lst[14]
        hob1_st1['d_1899'] = hob1_st1['d_1899']/div_coeff_lst[15]
        hob1_st1['d_1900'] = hob1_st1['d_1900']/div_coeff_lst[16]
        hob1_st1['d_1901'] = hob1_st1['d_1901']/div_coeff_lst[17]
        hob1_st1['d_1902'] = hob1_st1['d_1902']/div_coeff_lst[18]
        hob1_st1['d_1903'] = hob1_st1['d_1903']/div_coeff_lst[19]
        hob1_st1['d_1904'] = hob1_st1['d_1904']/div_coeff_lst[20]
        hob1_st1['d_1905'] = hob1_st1['d_1905']/div_coeff_lst[21]
        hob1_st1['d_1906'] = hob1_st1['d_1906']/div_coeff_lst[22]
        hob1_st1['d_1907'] = hob1_st1['d_1907']/div_coeff_lst[23]
        hob1_st1['d_1908'] = hob1_st1['d_1908']/div_coeff_lst[24]
        hob1_st1['d_1909'] = hob1_st1['d_1909']/div_coeff_lst[25]
        hob1_st1['d_1910'] = hob1_st1['d_1910']/div_coeff_lst[26]
        hob1_st1['d_1911'] = hob1_st1['d_1911']/div_coeff_lst[27]
        hob1_st1['d_1912'] = hob1_st1['d_1912']/div_coeff_lst[28]
        hob1_st1['d_1913'] = hob1_st1['d_1913']/div_coeff_lst[29]

        #Transform train data
        catcols = ['store_id', 'dept_id']
        dt_st_dep_n = pd.melt(dt_st_dep_n, 
                            id_vars = catcols, 
                            value_vars = [col for col in dt_st_dep_n.columns if col.startswith('d_')], 
                            var_name = 'd', 
                            value_name = 'sales')
        dt_st_dep_n = dt_st_dep_n.merge(cal, on= "d", copy = False)
        df_train = dt_st_dep_n

        lags = [7, 28]
        lag_cols = [f"lag_{lag}" for lag in lags ]
        for lag, lag_col in zip(lags, lag_cols):
            df_train[lag_col] = df_train['sales'].shift(lag)
        wins = [7, 28]
        for win in wins :
            for lag,lag_col in zip(lags, lag_cols):
                df_train[f"rmean_{lag}_{win}"] = df_train[lag_col].transform(lambda x : x.rolling(win).mean()) 
        df_train = df_train.drop(['store_id', 'dept_id', 'd', 'date', 'wm_yr_wk', 'weekday'], axis=1)
        df_train.dropna(inplace = True)

        y = df_train['sales']
        if (s_id.split('_')[0] == 'CA'):
            X_train = df_train.drop(['sales', 'snap_TX', 'snap_WI'], axis=1)
        elif (s_id.split('_')[0] == 'TX'):
            X_train = df_train.drop(['sales', 'snap_CA', 'snap_WI'], axis=1)
        else:
            X_train = df_train.drop(['sales', 'snap_TX', 'snap_CA'], axis=1)


        #Define and fit regressor
        gbr = GradientBoostingRegressor()
        grid_values_gbr = {'loss': ['ls','lad','huber'],'n_estimators': [50, 100, 250, 500], 
                           'max_depth':[3, 5, 7], 'learning_rate':[0.05, 0.075, 0.1],
                          'criterion': ['mse']}
        grid_clf_gbr = GridSearchCV(gbr, param_grid = grid_values_gbr, scoring = 'neg_root_mean_squared_error', cv =5)
        grid_clf_gbr.fit(X_train, y)
        gbr = grid_clf_gbr.best_estimator_
        gbr.fit(X_train,y)

        # Preparing dataset for prediction
        dt_pred = dt_st_dep.loc[(dt_st_dep['dept_id'] == d_id) & (dt_st_dep['store_id'] == s_id)]
        dt_pred = pd.melt(dt_pred, 
                            id_vars = catcols, 
                            value_vars = [col for col in dt_pred.columns if col.startswith('d_')], 
                            var_name = 'd', 
                            value_name = 'sales')
        for i in range(28):
            j=i+1914
            d = 'd_'+str(j)
            app_list = [s_id, d_id, d, -1]
            dt_pred.loc[len(dt_pred)] = app_list
        dt_pred = dt_pred.merge(future, on= "d", copy = False)
        dt_pred = dt_pred.drop(['store_id', 'dept_id', 'd', 'date', 'wm_yr_wk', 'weekday'], axis=1)

        #Getting features and predicting values for 1st week
        lags = [7, 28]
        lag_cols = [f"lag_{lag}" for lag in lags ]
        for lag, lag_col in zip(lags, lag_cols):
            dt_pred[lag_col] = dt_pred['sales'].shift(lag)
        wins = [7, 28]
        for win in wins :
            for lag,lag_col in zip(lags, lag_cols):
                dt_pred[f"rmean_{lag}_{win}"] = dt_pred[lag_col].transform(lambda x : x.rolling(win).mean()) 
        if (s_id.split('_')[0] == 'CA'):
            X_test_tmp = dt_pred.drop(['sales', 'snap_TX', 'snap_WI'], axis=1)
        elif (s_id.split('_')[0] == 'TX'):
            X_test_tmp = dt_pred.drop(['sales', 'snap_CA', 'snap_WI'], axis=1)
        else:
            X_test_tmp = dt_pred.drop(['sales', 'snap_TX', 'snap_CA'], axis=1)
        X_test_curr = X_test_tmp[1913:1920]
        y_pred_curr = gbr.predict(X_test_curr)
        y_pred_curr_lst = list(y_pred_curr)

        #Setting values for 1st week
        dt_pred.loc[1913, 'sales']=y_pred_curr_lst[0]
        dt_pred.loc[1914, 'sales']=y_pred_curr_lst[1]
        dt_pred.loc[1915, 'sales']=y_pred_curr_lst[2]
        dt_pred.loc[1916, 'sales']=y_pred_curr_lst[3]
        dt_pred.loc[1917, 'sales']=y_pred_curr_lst[4]
        dt_pred.loc[1918, 'sales']=y_pred_curr_lst[5]
        dt_pred.loc[1919, 'sales']=y_pred_curr_lst[6]

        #Getting features and predicting values for 2nd week
        lags = [7, 28]
        lag_cols = [f"lag_{lag}" for lag in lags ]
        for lag, lag_col in zip(lags, lag_cols):
            dt_pred[lag_col] = dt_pred['sales'].shift(lag)
        wins = [7, 28]
        for win in wins :
            for lag,lag_col in zip(lags, lag_cols):
                dt_pred[f"rmean_{lag}_{win}"] = dt_pred[lag_col].transform(lambda x : x.rolling(win).mean()) 
        if (s_id.split('_')[0] == 'CA'):
            X_test_tmp = dt_pred.drop(['sales', 'snap_TX', 'snap_WI'], axis=1)
        elif (s_id.split('_')[0] == 'TX'):
            X_test_tmp = dt_pred.drop(['sales', 'snap_CA', 'snap_WI'], axis=1)
        else:
            X_test_tmp = dt_pred.drop(['sales', 'snap_TX', 'snap_CA'], axis=1)
        X_test_curr = X_test_tmp[1920:1927]
        y_pred_curr = gbr.predict(X_test_curr)
        y_pred_curr_lst = list(y_pred_curr)

        #Setting values for 2nd week
        dt_pred.loc[1920, 'sales']=y_pred_curr_lst[0]
        dt_pred.loc[1921, 'sales']=y_pred_curr_lst[1]
        dt_pred.loc[1922, 'sales']=y_pred_curr_lst[2]
        dt_pred.loc[1923, 'sales']=y_pred_curr_lst[3]
        dt_pred.loc[1924, 'sales']=y_pred_curr_lst[4]
        dt_pred.loc[1925, 'sales']=y_pred_curr_lst[5]
        dt_pred.loc[1926, 'sales']=y_pred_curr_lst[6]

        # Getting features and predicting values for 3rd week
        lags = [7, 28]
        lag_cols = [f"lag_{lag}" for lag in lags ]
        for lag, lag_col in zip(lags, lag_cols):
            dt_pred[lag_col] = dt_pred['sales'].shift(lag)
        wins = [7, 28]
        for win in wins :
            for lag,lag_col in zip(lags, lag_cols):
                dt_pred[f"rmean_{lag}_{win}"] = dt_pred[lag_col].transform(lambda x : x.rolling(win).mean()) 
        if (s_id.split('_')[0] == 'CA'):
            X_test_tmp = dt_pred.drop(['sales', 'snap_TX', 'snap_WI'], axis=1)
        elif (s_id.split('_')[0] == 'TX'):
            X_test_tmp = dt_pred.drop(['sales', 'snap_CA', 'snap_WI'], axis=1)
        else:
            X_test_tmp = dt_pred.drop(['sales', 'snap_TX', 'snap_CA'], axis=1)
        X_test_curr = X_test_tmp[1927:1934]
        y_pred_curr = gbr.predict(X_test_curr)
        y_pred_curr_lst = list(y_pred_curr)

        # Setting values for 3rd week
        dt_pred.loc[1927, 'sales']=y_pred_curr_lst[0]
        dt_pred.loc[1928, 'sales']=y_pred_curr_lst[1]
        dt_pred.loc[1929, 'sales']=y_pred_curr_lst[2]
        dt_pred.loc[1930, 'sales']=y_pred_curr_lst[3]
        dt_pred.loc[1931, 'sales']=y_pred_curr_lst[4]
        dt_pred.loc[1932, 'sales']=y_pred_curr_lst[5]
        dt_pred.loc[1933, 'sales']=y_pred_curr_lst[6]

        # Getting features and predicting values for 4th week
        lags = [7, 28]
        lag_cols = [f"lag_{lag}" for lag in lags ]
        for lag, lag_col in zip(lags, lag_cols):
            dt_pred[lag_col] = dt_pred['sales'].shift(lag)
        wins = [7, 28]
        for win in wins :
            for lag,lag_col in zip(lags, lag_cols):
                dt_pred[f"rmean_{lag}_{win}"] = dt_pred[lag_col].transform(lambda x : x.rolling(win).mean()) 
        if (s_id.split('_')[0] == 'CA'):
            X_test_tmp = dt_pred.drop(['sales', 'snap_TX', 'snap_WI'], axis=1)
        elif (s_id.split('_')[0] == 'TX'):
            X_test_tmp = dt_pred.drop(['sales', 'snap_CA', 'snap_WI'], axis=1)
        else:
            X_test_tmp = dt_pred.drop(['sales', 'snap_TX', 'snap_CA'], axis=1)
        X_test_curr = X_test_tmp[1934:1941]
        y_pred_curr = gbr.predict(X_test_curr)
        y_pred_curr_lst = list(y_pred_curr)

        #Setting the predicted values for 4th week
        dt_pred.loc[1934, 'sales']=y_pred_curr_lst[0]
        dt_pred.loc[1935, 'sales']=y_pred_curr_lst[1]
        dt_pred.loc[1936, 'sales']=y_pred_curr_lst[2]
        dt_pred.loc[1937, 'sales']=y_pred_curr_lst[3]
        dt_pred.loc[1938, 'sales']=y_pred_curr_lst[4]
        dt_pred.loc[1939, 'sales']=y_pred_curr_lst[5]
        dt_pred.loc[1940, 'sales']=y_pred_curr_lst[6]

        #Plotting predicted and actual values
        dt_pred_temp = dt_pred.tail(28)
        y_pred_lst = dt_pred_temp['sales'].values.tolist()
        
        #Generating individual series predictions based on aggregated predictions and historical proportions
        hob1_st1['d_1886'] = hob1_st1['d_1886']*y_pred_lst[0]
        hob1_st1['d_1887'] = hob1_st1['d_1887']*y_pred_lst[1]
        hob1_st1['d_1888'] = hob1_st1['d_1888']*y_pred_lst[2]
        hob1_st1['d_1889'] = hob1_st1['d_1889']*y_pred_lst[3]
        hob1_st1['d_1890'] = hob1_st1['d_1890']*y_pred_lst[4]
        hob1_st1['d_1891'] = hob1_st1['d_1891']*y_pred_lst[5]
        hob1_st1['d_1892'] = hob1_st1['d_1892']*y_pred_lst[6]
        hob1_st1['d_1893'] = hob1_st1['d_1893']*y_pred_lst[7]
        hob1_st1['d_1894'] = hob1_st1['d_1894']*y_pred_lst[8]
        hob1_st1['d_1895'] = hob1_st1['d_1895']*y_pred_lst[9]
        hob1_st1['d_1896'] = hob1_st1['d_1896']*y_pred_lst[10]
        hob1_st1['d_1897'] = hob1_st1['d_1897']*y_pred_lst[11]
        hob1_st1['d_1898'] = hob1_st1['d_1898']*y_pred_lst[12]
        hob1_st1['d_1899'] = hob1_st1['d_1899']*y_pred_lst[13]
        hob1_st1['d_1900'] = hob1_st1['d_1900']*y_pred_lst[14]
        hob1_st1['d_1901'] = hob1_st1['d_1901']*y_pred_lst[15]
        hob1_st1['d_1902'] = hob1_st1['d_1902']*y_pred_lst[16]
        hob1_st1['d_1903'] = hob1_st1['d_1903']*y_pred_lst[17]
        hob1_st1['d_1904'] = hob1_st1['d_1904']*y_pred_lst[18]
        hob1_st1['d_1905'] = hob1_st1['d_1905']*y_pred_lst[19]
        hob1_st1['d_1906'] = hob1_st1['d_1906']*y_pred_lst[20]
        hob1_st1['d_1907'] = hob1_st1['d_1907']*y_pred_lst[21]
        hob1_st1['d_1908'] = hob1_st1['d_1908']*y_pred_lst[22]
        hob1_st1['d_1909'] = hob1_st1['d_1909']*y_pred_lst[23]
        hob1_st1['d_1910'] = hob1_st1['d_1910']*y_pred_lst[24]
        hob1_st1['d_1911'] = hob1_st1['d_1911']*y_pred_lst[25]
        hob1_st1['d_1912'] = hob1_st1['d_1912']*y_pred_lst[26]
        hob1_st1['d_1913'] = hob1_st1['d_1913']*y_pred_lst[27]
        
        #Adding current predictions to submission file
        df_sub = pd.concat([df_sub, hob1_st1], axis=0, sort=False)

CA_1 HOBBIES_1
CA_1 HOBBIES_2
CA_1 HOUSEHOLD_1
CA_1 HOUSEHOLD_2
CA_1 FOODS_1
CA_1 FOODS_2
CA_1 FOODS_3
CA_2 HOBBIES_1
CA_2 HOBBIES_2
CA_2 HOUSEHOLD_1
CA_2 HOUSEHOLD_2
CA_2 FOODS_1
CA_2 FOODS_2
CA_2 FOODS_3
CA_3 HOBBIES_1
CA_3 HOBBIES_2
CA_3 HOUSEHOLD_1
CA_3 HOUSEHOLD_2
CA_3 FOODS_1
CA_3 FOODS_2
CA_3 FOODS_3
CA_4 HOBBIES_1
CA_4 HOBBIES_2
CA_4 HOUSEHOLD_1
CA_4 HOUSEHOLD_2
CA_4 FOODS_1
CA_4 FOODS_2
CA_4 FOODS_3
TX_1 HOBBIES_1
TX_1 HOBBIES_2
TX_1 HOUSEHOLD_1
TX_1 HOUSEHOLD_2
TX_1 FOODS_1
TX_1 FOODS_2
TX_1 FOODS_3
TX_2 HOBBIES_1
TX_2 HOBBIES_2
TX_2 HOUSEHOLD_1
TX_2 HOUSEHOLD_2
TX_2 FOODS_1
TX_2 FOODS_2
TX_2 FOODS_3
TX_3 HOBBIES_1
TX_3 HOBBIES_2
TX_3 HOUSEHOLD_1
TX_3 HOUSEHOLD_2
TX_3 FOODS_1
TX_3 FOODS_2
TX_3 FOODS_3
WI_1 HOBBIES_1
WI_1 HOBBIES_2
WI_1 HOUSEHOLD_1
WI_1 HOUSEHOLD_2
WI_1 FOODS_1
WI_1 FOODS_2
WI_1 FOODS_3
WI_2 HOBBIES_1
WI_2 HOBBIES_2
WI_2 HOUSEHOLD_1
WI_2 HOUSEHOLD_2
WI_2 FOODS_1
WI_2 FOODS_2
WI_2 FOODS_3
WI_3 HOBBIES_1
WI_3 HOBBIES_2
WI_3 HOUSEHOLD_1
WI_3 HOUSEHOLD_2
WI_3 FOOD

In [14]:
df_sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30490 entries, 0 to 30489
Data columns (total 31 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        30490 non-null  object 
 1   dept_id   30490 non-null  object 
 2   store_id  30490 non-null  object 
 3   d_1886    30490 non-null  float64
 4   d_1887    30490 non-null  float64
 5   d_1888    30490 non-null  float64
 6   d_1889    30490 non-null  float64
 7   d_1890    30490 non-null  float64
 8   d_1891    30490 non-null  float64
 9   d_1892    30490 non-null  float64
 10  d_1893    30490 non-null  float64
 11  d_1894    30490 non-null  float64
 12  d_1895    30490 non-null  float64
 13  d_1896    30490 non-null  float64
 14  d_1897    30490 non-null  float64
 15  d_1898    30490 non-null  float64
 16  d_1899    30490 non-null  float64
 17  d_1900    30490 non-null  float64
 18  d_1901    30490 non-null  float64
 19  d_1902    30490 non-null  float64
 20  d_1903    30490 non-null  fl

In [15]:
df_sub_2 = df_sub.drop(['store_id', 'dept_id'], axis=1)  
COLUMN_NAMES = ['id', 'F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13','F14','F15','F16','F17','F18','F19',
               'F20', 'F21','F22','F23','F24','F25','F26','F27','F28']
df_sub_2.columns = COLUMN_NAMES
df_sub_c = df_sub_2.copy()
df_sub_c["id"] = df_sub_c["id"].str.replace("validation$", "evaluation")
df_sub_2 = pd.concat([df_sub_2, df_sub_c], axis=0, sort=False)
df_sub_2.to_csv("submission_gbt_grid.csv",index=False)

In [16]:
df_sub_2.id.nunique(), df_sub_2["id"].str.contains("validation$").sum()

(60980, 30490)

In [17]:
df_sub_2.shape

(60980, 29)